# Bottom Up Tree Search Synthesizer and SMT Based Verification Oracle

The Bottom Up Tree Search Synthesizer accepts a CFG written in sympy expressions and synthesizes sympy expressions, while the SMT Based Verification Oracle operates on z3 expressions.

## Initializing `verification_oracle_instance`

In [1]:
from z3 import *

from verification_oracle import verification_oracle


z3_x = Int('x')
z3_y = Int('y')

z3_input_variable_list = [z3_x, z3_y]
z3_function_declaration = max2 = Function('max2', IntSort(), IntSort(), IntSort())
z3_constraint = And(max2(z3_x, z3_y) >= z3_x, max2(z3_x, z3_y) >= z3_y, Or(max2(z3_y, z3_x) == z3_x, max2(z3_y, z3_x) == z3_y))

verification_oracle_instance = verification_oracle(z3_input_variable_list, z3_function_declaration, z3_constraint)
next(verification_oracle_instance)

## Initializing `bottom_up_tree_search_instance`

In [2]:
from sympy.core.add import Add
from sympy.core.function import Function
from sympy.core.mul import Mul
from sympy.core.numbers import Integer
from sympy.core.relational import Equality, GreaterThan, LessThan
from sympy.core.symbol import Symbol
from sympy.logic.boolalg import And, Or, Not
from sympy.functions.elementary.piecewise import ExprCondPair, Piecewise


S = Symbol('S')
B = Symbol('B')
x = Symbol('x')
y = Symbol('y')

non_terminals = { S, B }
terminals = { x, y }
# non_terminals to a set of tuples containing the production_rule in reverse_polish_notation
production_rules = {
  S: {
    # x
    (x,),
    # y
    (y,),
    # 0
    (Integer(0),),
    # 1
    (Integer(1),),
    # + S S
    (S, S, (Add, 2)),
    # - S S
    (S, Integer(-1), S, (Mul, 2), (Add, 2)),
    # ite B S S
    (S, B, (ExprCondPair, 2), S, True, (ExprCondPair, 2), (Piecewise, 2)),
  },
  B: {
    # and B B
    (B, B, (And, 2)),
    # or B B
    (B, B, (Or, 2)),
    # not B
    (B, (Not, 1)),
    # <= S S
    (S, S, (LessThan, 2)),
    # = S S
    (S, S, (Equality, 2)),
    # >= S S
    (S, S, (GreaterThan, 2)),
  }
}
start_symbol = S

function_declaration = max2 = Function('max2')
constraint = And(GreaterThan(max2(x, y), x), GreaterThan(max2(x, y), y), Or(Equality(max2(y, x), x), Equality(max2(y, x), y)))

In [3]:
from bottom_up_tree_search import bottom_up_tree_search

bottom_up_tree_search_instance = bottom_up_tree_search(
    non_terminals,
    terminals,
    production_rules,
    start_symbol,
    function_declaration,
    constraint
)

## Conversion between sympy expressions and z3 expressions

In [4]:
from sympy_expr_to_z3_expr_ref import sympy_expr_to_z3_expr_ref
from z3_expr_ref_to_sympy_expr import z3_expr_ref_to_sympy_expr

sympy_symbols_to_z3_expr_ref = {
    x: z3_x,
    y: z3_y
}

z3_expr_ref_to_sympy_symbols = {
    z3_x: x,
    z3_y: y
}

## Program Synthesis and Verification

In [5]:
from frozendict import frozendict

In [6]:
sympy_expr = next(bottom_up_tree_search_instance)

In [7]:
print(f'sympy_expr {sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = bottom_up_tree_search_instance.send(counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

sympy_expr x
skipped 1
z3_counterexample_input {x: -1, y: 0}


In [8]:
print(f'sympy_expr {sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = bottom_up_tree_search_instance.send(counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

sympy_expr 0
z3_counterexample_input {x: -1, y: -1}


In [9]:
print(f'sympy_expr {sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = bottom_up_tree_search_instance.send(counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

sympy_expr y
skipped 2*x
skipped x + 1
skipped x + y
skipped 2
skipped y + 1
skipped 2*y
skipped x - 1
skipped x - y
skipped 1 - x
skipped 1 - y
skipped -x
skipped -1
skipped -y
skipped -x + y
skipped y - 1
skipped 3*x
skipped 2*x + 1
skipped 2*x + y
skipped x + 2
z3_counterexample_input {x: 0, y: -1}


In [10]:
print(f'sympy_expr {sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = bottom_up_tree_search_instance.send(counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

sympy_expr x + y + 1
skipped x + 2*y
skipped 2*x - 1
skipped 2*x - y
skipped x - y + 1
skipped x + y - 1
skipped 3
skipped y + 2
skipped 2*y + 1
skipped 2 - x
skipped 2 - y
skipped -x + y + 1
skipped 3*y
skipped -x + 2*y
skipped 2*y - 1
skipped x - 2
skipped x - y - 1
skipped x - 2*y
skipped 1 - 2*x
skipped -x - y + 1
skipped 1 - 2*y
skipped -2*x
skipped -x - 1
skipped -x - y
skipped -2
skipped -y - 1
skipped -2*y
skipped -2*x + y
skipped -x + y - 1
skipped y - 2
skipped 4*x
skipped 3*x + 1
skipped 3*x + y
skipped 2*x + 2
skipped 2*x + y + 1
skipped 2*x + 2*y
skipped 3*x - 1
skipped 3*x - y
skipped 2*x - y + 1
skipped 2*x + y - 1
skipped x + 3
skipped x + y + 2
skipped x + 2*y + 1
skipped x - y + 2
skipped x + 3*y
skipped x + 2*y - 1
skipped 2*x - 2
skipped 2*x - y - 1
skipped 2*x - 2*y
skipped x - 2*y + 1
skipped x + y - 2
skipped 4
skipped y + 3
skipped 2*y + 2
skipped 3 - x
skipped 3 - y
skipped -x + y + 2
skipped 3*y + 1
skipped -x + 2*y + 1
skipped 2 - 2*x
skipped -x - y + 2
skipp

In [11]:
print(f'sympy_expr {sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = bottom_up_tree_search_instance.send(counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

sympy_expr Piecewise((0, Eq(x, 0)), (y, True))
skipped Piecewise((y, Eq(x, y)), (1, True))
skipped Piecewise((y, Eq(x, y)), (0, True))
skipped Piecewise((x, Eq(y, 1)), (1, True))
skipped Piecewise((x, Eq(y, 1)), (0, True))
skipped Piecewise((x, Eq(y, 1)), (y, True))
skipped Piecewise((x, Eq(y, 0)), (1, True))
skipped Piecewise((x, Eq(y, 0)), (0, True))
skipped Piecewise((x, Eq(y, 0)), (y, True))
skipped Piecewise((x, x >= 1), (1, True))
skipped Piecewise((x, x >= 1), (0, True))
skipped Piecewise((x, x >= 1), (y, True))
skipped Piecewise((x, x >= 0), (1, True))
skipped Piecewise((x, x >= 0), (0, True))
z3_counterexample_input {x: 2, y: 1}


In [12]:
print(f'sympy_expr {sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = bottom_up_tree_search_instance.send(counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

sympy_expr Piecewise((x, x >= 0), (y, True))
skipped Piecewise((x, x >= y), (1, True))
skipped Piecewise((x, x >= y), (0, True))
z3_counterexample_input {x: -1, y: -2}


In [13]:
print(f'sympy_expr {sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = bottom_up_tree_search_instance.send(counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

sympy_expr Piecewise((x, x >= y), (y, True))
z3_counterexample_input None
